In [4]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
import numpy as np
from PIL import Image as PILImage

class RGBDSaver(Node):
    def __init__(self):
        super().__init__('rgbd_saver')
        self.bridge = CvBridge()
        self.DIR = "./"  # Change this to your desired directory
    
    def save_rgbd(self):
        rgb_message_wrist = self.wait_for_message("camera/camera/color/image_rect_raw", Image, timeout=5)
        depth_message_wrist = self.wait_for_message("camera/camera/aligned_depth_to_color/image_raw", Image, timeout=5)

        if rgb_message_wrist is None or depth_message_wrist is None:
            self.get_logger().error("Failed to receive images in time.")
            return None, None

        rgb_image_wrist = self.bridge.imgmsg_to_cv2(rgb_message_wrist, desired_encoding="bgr8")
        depth_image_wrist = self.bridge.imgmsg_to_cv2(depth_message_wrist, desired_encoding="16UC1")
        
        rgb_image_wrist = PILImage.fromarray(cv2.cvtColor(rgb_image_wrist, cv2.COLOR_BGR2RGB))
        depth_image_wrist = PILImage.fromarray(depth_image_wrist)
        
        rgb_dir = f"{self.DIR}/mug/demo_wrist_rgb.png"
        depth_dir = f"{self.DIR}/mug/demo_wrist_depth.png"
        
        rgb_image_wrist.save(rgb_dir)
        depth_image_wrist.save(depth_dir)

        return rgb_dir, depth_dir
    
    def wait_for_message(self, topic, msg_type, timeout=5):
        future = rclpy.Future()

        def callback(msg):
            if not future.done():
                future.set_result(msg)

        sub = self.create_subscription(msg_type, topic, callback, 10)
        self.get_logger().info(f"Waiting for message on {topic}...")

        start_time = self.get_clock().now().seconds_nanoseconds()[0]
        
        while rclpy.ok():
            rclpy.spin_once(self, timeout_sec=0.1)
            if future.done():
                return future.result()
            if self.get_clock().now().seconds_nanoseconds()[0] - start_time > timeout:
                self.get_logger().warning(f"Timeout while waiting for message on {topic}")
                return None


In [8]:

def main():
    rclpy.init()
    node = RGBDSaver()
    
    try:
        rgb_path, depth_path = node.save_rgbd()
        if rgb_path and depth_path:
            node.get_logger().info(f"RGB saved at: {rgb_path}")
            node.get_logger().info(f"Depth saved at: {depth_path}")
    finally:
        node.destroy_node()
        rclpy.shutdown()

if __name__ == '__main__':
    main()


[INFO] [1741363905.122369031] [rgbd_saver]: Waiting for message on camera/camera/color/image_rect_raw...
[INFO] [1741363905.146981477] [rgbd_saver]: Waiting for message on camera/camera/aligned_depth_to_color/image_raw...
[INFO] [1741363905.349250351] [rgbd_saver]: RGB saved at: .//mug/demo_wrist_rgb.png
[INFO] [1741363905.351373659] [rgbd_saver]: Depth saved at: .//mug/demo_wrist_depth.png


In [ ]:
from romatch import roma_indoor
import cv2
import matplotlib.pyplot as plt

def warp_corners_and_draw_matches(ref_points, dst_points, img1, img2):
    # Calculate the Homography matrix
    H, mask = cv2.findHomography(ref_points, dst_points, cv2.USAC_MAGSAC, 3.5, maxIters=1_000, confidence=0.999)
    mask = mask.flatten()

    # Get corners of the first image (image1)
    h, w = img1.shape[:2]
    corners_img1 = np.array([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]], dtype=np.float32).reshape(-1, 1, 2)

    # Warp corners to the second image (image2) space
    warped_corners = cv2.perspectiveTransform(corners_img1, H)

    # Draw the warped corners in image2
    img2_with_corners = img2.copy()
    for i in range(len(warped_corners)):
        start_point = tuple(warped_corners[i-1][0].astype(int))
        end_point = tuple(warped_corners[i][0].astype(int))
        cv2.line(img2_with_corners, start_point, end_point, (0, 255, 0), 4)  # Using solid green for corners

    # Prepare keypoints and matches for drawMatches function
    keypoints1 = [cv2.KeyPoint(p[0], p[1], 5) for p in ref_points]
    keypoints2 = [cv2.KeyPoint(p[0], p[1], 5) for p in dst_points]
    matches = [cv2.DMatch(i,i,0) for i in range(len(mask)) if mask[i]]

    # Draw inlier matches
    img_matches = cv2.drawMatches(img1, keypoints1, img2_with_corners, keypoints2, matches, None,
                                  matchColor=(0, 255, 0), flags=2)


img1_path = "/home/yilong/ObAct/robot_control/mug/demo_wrist_rgb.png"
img2_path = "/home/yilong/ObAct/robot_control/mug/live_wrist_rgb_3.png"
img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

roma_model = roma_indoor(device="cuda")
# Match
warp, certainty = roma_model.match(img1_path, img2_path, device="cuda")
# Sample matches for estimation
matches, certainty = roma_model.sample(warp, certainty)
# Convert to pixel coordinates (RoMa produces matches in [-1,1]x[-1,1])
kptsA, kptsB = roma_model.to_pixel_coordinates(matches, 848, 480, 848, 480)
# Find a fundamental matrix (or anything else of interest)
F, mask = cv2.findFundamentalMat(
    kptsA.cpu().numpy(), kptsB.cpu().numpy(), ransacReprojThreshold=0.2, method=cv2.USAC_MAGSAC, confidence=0.999999, maxIters=10000
)

canvas = warp_corners_and_draw_matches(kptsA, kptsB, img1, img2)
plt.figure(figsize=(12,12))
plt.imshow(canvas), plt.show()

/home/yilong/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/yilong/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Using coarse resolution (560, 560), and upsample res (864, 864)


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'findHomography'
> Overload resolution failed:
>  - srcPoints is not a numpy array, neither a scalar
>  - findHomography() takes at most 4 arguments (6 given)
>  - Expected Ptr<cv::UMat> for argument 'srcPoints'
>  - findHomography() takes at most 4 arguments (6 given)


In [10]:
import cv2
import numpy as np
import torch
# from romatch import roma_indoor
import matplotlib.pyplot as plt

# # Initialize RoMa Model
# roma_model = roma_indoor(device="cuda")

# Load images
img1_path = "/home/yilong/ObAct/robot_control/pan/demo_wrist_rgb.png"
img1_mask_path = "/home/yilong/ObAct/robot_control/mug/live_wrist_rgb_3.png"

# Read images
img1 = cv2.imread(img1_path)
img2 = cv2.imread(img1_path)
img1_mask = cv2.imread(img1_mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask as grayscale

# Normalize mask to be in range [0,1] for multiplication
img1_mask = img1_mask / 255.0  # Convert to float range [0,1]

# Apply mask to the image
img1 = (img1 * img1_mask[:, :, None]).astype(np.uint8)  # Preserve 3 channels

# cv2.imwrite("/home/yilong/ObAct/robot_control/pan/demo_wrist_masked_rgb.png", img1)

# # Match features
# warp, certainty = roma_model.match(img1_mask_rgb_path, img2_path, device="cuda")

# # Sample matches
# matches, certainty = roma_model.sample(warp, certainty)

# # Convert to pixel coordinates
# kptsA, kptsB = roma_model.to_pixel_coordinates(matches, 848, 480, 848, 480)

# # Convert tensors to NumPy arrays
# kptsA = kptsA.cpu().numpy()
# kptsB = kptsB.cpu().numpy()



In [11]:
def warp_corners_and_draw_matches(ref_points, dst_points, img1, img2):
    # Calculate the Homography matrix
    H, mask = cv2.findHomography(ref_points, dst_points, cv2.USAC_MAGSAC, 3.5, maxIters=1_000, confidence=0.999)
    mask = mask.flatten()

    # Get corners of the first image (image1)
    h, w = img1.shape[:2]
    corners_img1 = np.array([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]], dtype=np.float32).reshape(-1, 1, 2)

    # Warp corners to the second image (image2) space
    warped_corners = cv2.perspectiveTransform(corners_img1, H)

    # Draw the warped corners in image2
    img2_with_corners = img2.copy()
    for i in range(len(warped_corners)):
        start_point = tuple(warped_corners[i-1][0].astype(int))
        end_point = tuple(warped_corners[i][0].astype(int))
        cv2.line(img2_with_corners, start_point, end_point, (0, 255, 0), 4)  # Using solid green for corners

    # Prepare keypoints and matches for drawMatches function
    keypoints1 = [cv2.KeyPoint(p[0], p[1], 5) for p in ref_points]
    keypoints2 = [cv2.KeyPoint(p[0], p[1], 5) for p in dst_points]
    matches = [cv2.DMatch(i,i,0) for i in range(len(mask)) if mask[i]]

    # Draw inlier matches
    img_matches = cv2.drawMatches(img1, keypoints1, img2_with_corners, keypoints2, matches, None,
                                  matchColor=(0, 255, 0), flags=2)

    return img_matches


canvas = warp_corners_and_draw_matches(kptsA, kptsB, img1, img2)
plt.figure(figsize=(12,12))
plt.imshow(canvas), plt.show()

error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'findHomography'
> Overload resolution failed:
>  - srcPoints is not a numpy array, neither a scalar
>  - findHomography() takes at most 4 arguments (6 given)
>  - Expected Ptr<cv::UMat> for argument 'srcPoints'
>  - findHomography() takes at most 4 arguments (6 given)


In [1]:
from lightglue import LightGlue, SuperPoint, DISK, SIFT, ALIKED, DoGHardNet
from lightglue.utils import load_image, rbd

# SuperPoint+LightGlue
extractor = SuperPoint(max_num_keypoints=2048).eval().cuda()  # load the extractor
matcher = LightGlue(features='superpoint').eval().cuda()  # load the matcher


# load each image as a torch.Tensor on GPU with shape (3,H,W), normalized in [0,1]
image0 = load_image(img1_mask_rgb_path).cuda()
image1 = load_image(img2_path).cuda()
# extract local features
feats0 = extractor.extract(image0)  # auto-resize the image, disable with resize=None
feats1 = extractor.extract(image1)

# match the features
matches01 = matcher({'image0': feats0, 'image1': feats1})
feats0, feats1, matches01 = [rbd(x) for x in [feats0, feats1, matches01]]  # remove batch dimension
matches = matches01['matches']  # indices with shape (K,2)
matches = matches01['matches']  # indices with shape (K,2)
mkpts_0 = feats0['keypoints'][matches[..., 0]].cpu().numpy()  # coordinates in image #0, shape (K,2)
mkpts_1 = feats1['keypoints'][matches[..., 1]].cpu().numpy()  # coordinates in image #1, shape (K,2)

canvas = warp_corners_and_draw_matches(mkpts_0 , mkpts_1, img1, img2)
plt.figure(figsize=(12,12))
plt.imshow(canvas), plt.show()

/home/yilong/LightGlue/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


NameError: name 'img1_mask_rgb_path' is not defined

In [1]:
import rclpy
from rclpy.node import Node
import tf2_ros
import geometry_msgs.msg
import tf_transformations  # Converts Euler to quaternion

class TransformBroadcasterNode(Node):
    def __init__(self):
        super().__init__('tf_broadcaster')
        self.tf_broadcaster = tf2_ros.TransformBroadcaster(self)
        self.timer = self.create_timer(0.1, self.broadcast_transform)  # Publish at 10 Hz

    def broadcast_transform(self):
        t = geometry_msgs.msg.TransformStamped()

        # Set timestamp
        t.header.stamp = self.get_clock().now().to_msg()
        t.header.frame_id = "vx300s/base_link"  # Parent frame
        t.child_frame_id = "goal"  # Child frame

        # Set translation
        t.transform.translation.x = 0.22555483
        t.transform.translation.y = -0.00516697
        t.transform.translation.z = 0.2176612

        # Convert Euler angles to quaternion
        roll = 0.00933795
        pitch = 0.78332596
        yaw = 0.01755486
        q = tf_transformations.quaternion_from_euler(roll, pitch, yaw)

        # Set rotation
        t.transform.rotation.x = q[0]
        t.transform.rotation.y = q[1]
        t.transform.rotation.z = q[2]
        t.transform.rotation.w = q[3]

        # Publish transform
        self.tf_broadcaster.sendTransform(t)
        self.get_logger().info("Broadcasting transform from base_link to goal")

def main():
    rclpy.init()
    node = TransformBroadcasterNode()
    rclpy.spin(node)
    node.destroy_node()
    rclpy.shutdown()

if __name__ == '__main__':
    main()



[INFO] [1740761663.789704686] [tf_broadcaster]: Broadcasting transform from base_link to goal
[INFO] [1740761663.866858232] [tf_broadcaster]: Broadcasting transform from base_link to goal
[INFO] [1740761663.966837981] [tf_broadcaster]: Broadcasting transform from base_link to goal
[INFO] [1740761664.066879449] [tf_broadcaster]: Broadcasting transform from base_link to goal
[INFO] [1740761664.166853111] [tf_broadcaster]: Broadcasting transform from base_link to goal
[INFO] [1740761664.266817989] [tf_broadcaster]: Broadcasting transform from base_link to goal
[INFO] [1740761664.366776812] [tf_broadcaster]: Broadcasting transform from base_link to goal
[INFO] [1740761664.466833053] [tf_broadcaster]: Broadcasting transform from base_link to goal
[INFO] [1740761664.566949964] [tf_broadcaster]: Broadcasting transform from base_link to goal
[INFO] [1740761664.666824450] [tf_broadcaster]: Broadcasting transform from base_link to goal
[INFO] [1740761664.766896901] [tf_broadcaster]: Broadcasting

KeyboardInterrupt: 

In [8]:
import numpy as np
import open3d as o3d
import cv2
from PIL import Image as PILImage

# File paths
rgb_dir = "/home/yilong/ObAct/robot_control/mug/demo_wrist_rgb.png"
depth_dir = "/home/yilong/ObAct/robot_control/mug/demo_wrist_depth.png"
mask_dir = "/home/yilong/ObAct/robot_control/outputs/mask_0.png"

# Load images
rgb_image = np.array(PILImage.open(rgb_dir))
depth_image = np.array(PILImage.open(depth_dir))
mask = np.array(PILImage.open(mask_dir))
d405_intrinsic = np.load("/home/yilong/ObAct/robot_control/d405_intrinsic.npy")

# Convert RGB image to correct format if needed
if rgb_image.shape[-1] == 4:  # If RGBA
    rgb_image = rgb_image[..., :3]
rgb_image = rgb_image.astype(np.uint8)

# Ensure depth image is in correct format (assuming it's in mm)
depth_image = depth_image.astype(np.float32)

# Apply mask if needed
depth_image = depth_image * (mask > 0)  # Apply mask to depth

# Create RGBD image
color = o3d.geometry.Image(rgb_image)
depth = o3d.geometry.Image(depth_image)

rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color, 
    depth,
    depth_scale=1000.0,  # Adjust based on your depth image units (1000 if in mm)
    depth_trunc=3.0,     # Maximum depth in meters
    convert_rgb_to_intensity=False
)

# Extract parameters from K matrix
fx = d405_intrinsic[0, 0]  # f_x from K[0,0]
fy = d405_intrinsic[1, 1]  # f_y from K[1,1]
cx = d405_intrinsic[0, 2]  # c_x from K[0,2]
cy = d405_intrinsic[1, 2]  # c_y from K[1,2]

# Set camera intrinsics
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.set_intrinsics(
    width=rgb_image.shape[1],
    height=rgb_image.shape[0],
    fx=fx,
    fy=fy,
    cx=cx,
    cy=cy
)

# Create point cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    intrinsic
)

# Optional: Downsample the point cloud
pcd = pcd.voxel_down_sample(voxel_size=0.005)  # Adjust voxel size as needed

# Optional: Remove statistical outliers
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

# Visualize
o3d.visualization.draw_geometries([pcd],
    window_name="Point Cloud",
    width=800,
    height=600,
    point_show_normal=False
)

In [ ]:
import numpy as np
import open3d as o3d
import cv2
from PIL import Image as PILImage

# Calculate center of point cloud
points = np.asarray(pcd.points)
center = np.mean(points, axis=0)
print(f"Point cloud center: {center}")

def generate_hemisphere_points_with_orientations(self, radius=0.1, num_points=100):
    """Generate hemispherical sampling points with orientations around Z-axis"""
    points = np.asarray(self.pcd.points)
    self.center = np.mean(points, axis=0)
    
    sampling_data = []
    phi = np.pi * (3. - np.sqrt(5.))  # golden angle in radians
    
    for i in range(num_points):
        z = 1 - (i / float(num_points - 1)) * 2  # z from 1 to -1
        radius_at_z = np.sqrt(1 - z * z)
        theta = phi * i
        
        x = np.cos(theta) * radius_at_z
        y = np.sin(theta) * radius_at_z
        
        if z >= 0:  # Upper hemisphere
            # Position
            pos = np.array([x, y, z]) * radius + self.center
            
            # Orientation: Point towards center
            direction = self.center - pos  # Vector from position to center
            direction = direction / np.linalg.norm(direction)  # Normalize
            
            # Z-axis of the viewpoint (looking towards center)
            z_axis = direction
            
            # X-axis: Perpendicular to z_axis and global Z [0,0,1]
            global_z = np.array([0, 0, 1])
            x_axis = np.cross(global_z, z_axis)
            if np.linalg.norm(x_axis) > 0:  # Ensure not parallel to global Z
                x_axis = x_axis / np.linalg.norm(x_axis)
            else:
                x_axis = np.array([1, 0, 0])  # Default if parallel
            
            # Y-axis: Perpendicular to z_axis and x_axis
            y_axis = np.cross(z_axis, x_axis)
            y_axis = y_axis / np.linalg.norm(y_axis)
            
            # Rotation matrix
            R = np.column_stack((x_axis, y_axis, z_axis))
            
            # Convert to roll, pitch, yaw (in radians)
            yaw = np.arctan2(R[1, 0], R[0, 0])
            pitch = np.arctan2(-R[2, 0], np.sqrt(R[2, 1]**2 + R[2, 2]**2))
            roll = np.arctan2(R[2, 1], R[2, 2])
            
            sampling_data.append({
                'position': pos,
                'roll': roll,
                'pitch': pitch,
                'yaw': yaw,
                'rotation': R
            })
    
    return sampling_data

def transform_to_robot_frame(center):
    """Transform point cloud from camera frame to robot frame"""
    # Load hand-eye calibration matrix (camera to end-effector)
    cam_to_ee = np.load("/home/yilong/ObAct/robot_control/d405_extrinsic.npy")
    
    # Robot's end-effector pose in robot frame
    ee_to_robot = np.array([
        [6.07944607e-01, 6.35465678e-04, 7.93979188e-01, 2.52638927e-01],
        [-3.54942557e-03, 9.99991863e-01, 1.91742259e-03, -8.50579388e-04],
        [-7.93971508e-01, -3.98385675e-03, 6.07941915e-01, 2.82457341e-01],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]
    ])
    
    # Complete transformation: robot_frame = ee_to_robot * cam_to_ee * camera_frame
    transform = ee_to_robot @ cam_to_ee
    
    # Transform the point cloud
    pcd.transform(transform)
    
    # If center exists (after sampling), transform it too

    center_homogeneous = np.append(center, 1)  # Convert to homogeneous coordinates
    center = np.dot(transform, center_homogeneous)[:3]  # Transform and extract 3D point
    
    return center

center = transform_to_robot_frame(center)

# Generate sampling points
radius = 0.1  # 0.1 meters
sampling_points = generate_hemisphere_points(center, radius, num_points=100)

# Create point cloud for sampling points
sampling_pcd = o3d.geometry.PointCloud()
sampling_pcd.points = o3d.utility.Vector3dVector(sampling_points)
# Color sampling points red
sampling_pcd.paint_uniform_color([1.0, 0.0, 0.0])

# Create sphere at center for reference
center_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.005)
center_sphere.translate(center)
center_sphere.paint_uniform_color([0.0, 1.0, 0.0])  # Green color

# Create coordinate frame for reference
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.05, 
    origin=center
)

# Visualize everything
o3d.visualization.draw_geometries(
    [pcd, sampling_pcd, center_sphere, coord_frame],
    window_name="Point Cloud with Hemispherical Sampling",
    width=800,
    height=600,
    point_show_normal=False,
    mesh_show_wireframe=False
)

# Optional: Print some statistics
print(f"Number of original points: {len(pcd.points)}")
print(f"Number of sampling points: {len(sampling_points)}")
print(f"Radius used: {radius}m")

Point cloud center: [ 1.06546747 -0.5553738   0.19927063]
Number of original points: 938
Number of sampling points: 51
Radius used: 0.1m


In [18]:
sampling_points

array([[ 0.87415967, -0.94812974,  0.29760498],
       [ 0.85941304, -0.95014994,  0.3111141 ],
       [ 0.89501936, -0.95419035,  0.32481276],
       [ 0.86177102, -0.96025095,  0.34369012],
       [ 0.92536022, -0.96429136,  0.31630334],
       [ 0.82101353, -0.96631156,  0.31954293],
       [ 0.89297083, -0.97035196,  0.35757795],
       [ 0.8340791 , -0.97641257,  0.35461546],
       [ 0.93045287, -0.98045297,  0.34504895],
       [ 0.79879719, -0.98247318,  0.30072145],
       [ 0.87052154, -0.98651358,  0.37628288],
       [ 0.95519153, -0.99055398,  0.3085077 ],
       [ 0.80590698, -0.99257419,  0.34509342],
       [ 0.916773  , -0.99661459,  0.37197092],
       [ 0.8397723 , -1.0026752 ,  0.37977184],
       [ 0.95473072, -1.0067156 ,  0.33995087],
       [ 0.78528193, -1.0087358 ,  0.32103287],
       [ 0.89000347, -1.01277621,  0.38979555],
       [ 0.80923057, -1.01883681,  0.36779143],
       [ 0.93917062, -1.02287722,  0.36927007],
       [ 0.85659403, -1.02893782,  0.394

In [2]:
from interbotix_common_modules.common_robot.robot import robot_shutdown, robot_startup
from interbotix_xs_modules.xs_robot.arm import InterbotixManipulatorXS

bot = InterbotixManipulatorXS(
    robot_model='vx300s',
    group_name='arm',
    gripper_name='gripper',
    moving_time=2,
    accel_time=0.3
)

robot_startup()

print(bot.arm.get_ee_pose().copy())

[INFO] [1741372480.907300655] [interbotix_robot_manipulation]: Initialized InterbotixRobotNode!
[INFO] [1741372480.933746643] [interbotix_robot_manipulation]: 
	Robot Name: vx300s
	Robot Model: vx300s
[INFO] [1741372480.933926363] [interbotix_robot_manipulation]: Initialized InterbotixRobotXSCore!
[INFO] [1741372480.937282162] [interbotix_robot_manipulation]: 
	Arm Group Name: arm
	Moving Time: 2.00 seconds
	Acceleration Time: 0.30 seconds
	Drive Mode: Time-Based-Profile
[INFO] [1741372480.937445787] [interbotix_robot_manipulation]: Initialized InterbotixArmXSInterface!


[[ 6.07944607e-01  6.35465678e-04  7.93979188e-01  2.52638927e-01]
 [-3.54942557e-03  9.99991863e-01  1.91742259e-03 -8.50579388e-04]
 [-7.93971508e-01 -3.98385675e-03  6.07941915e-01  2.82457341e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


[INFO] [1741372481.439429198] [interbotix_robot_manipulation]: 
	Gripper Name: gripper
	Gripper Pressure: 50.0%
[INFO] [1741372481.439873856] [interbotix_robot_manipulation]: Initialized InterbotixGripperXSInterface!


In [3]:
print(bot.arm.get_ee_pose().copy())

[[ 6.07944607e-01  6.35465678e-04  7.93979188e-01  2.52638927e-01]
 [-3.54942557e-03  9.99991863e-01  1.91742259e-03 -8.50579388e-04]
 [-7.93971508e-01 -3.98385675e-03  6.07941915e-01  2.82457341e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
